In [ ]:
from google.colab import drive
import shutil
drive.mount('/content/drive/')
load_path = '/content/drive/My Drive/yuqing/'
save_path = '.'
extract_path = {
    'ILSVRC2012_img_val.tar': 'imgs',
    'ILSVRC2012_devkit_t12.tar.gz': '.'
}
files = ('ILSVRC2012_devkit_t12.tar.gz', 'ILSVRC2012_img_val.tar', 'labels.txt')
for file in files:
    shutil.copy2(load_path+file, save_path)
    if 'tar' in file:
        shutil.unpack_archive(file, extract_path[file])


Mounted at /content/drive/


In [1]:
"""
This script will demonstrate how to use a pretrained model, in PyTorch, 
to make predictions. Specifically, we will be using VGG16 with a cat 
image.
References used to make this script:
PyTorch pretrained models doc:
    http://pytorch.org/docs/master/torchvision/models.html
PyTorch image transforms example:
    http://pytorch.org/tutorials/beginner/data_loading_tutorial.html#transforms
Example code:
    http://blog.outcome.io/pytorch-quick-start-classifying-an-image/    
"""

import io
import os

from PIL import Image
import requests
import numpy as np
from tqdm import tqdm

from torch.autograd import Variable
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn
torch.manual_seed(1000)

# Random cat img taken from Google
IMG_URL = 'https://upload.wikimedia.org/wikipedia/commons/thumb/3/3a/Cat03.jpg/1200px-Cat03.jpg'
# Class labels used when training VGG as json, courtesy of the 'Example code' link above.
LABELS_URL = 'https://s3.amazonaws.com/outcome-blog/imagenet/labels.json'

# Let's get our class labels.
'''
Following two methods are for the readable prediction labels
'''
def get_rpred():
    response = requests.get(LABELS_URL)  # Make an HTTP GET request and store the response.
    labels = {int(key): value for key, value in response.json().items()}
    return labels
rpred = get_rpred()

def readable_pred(pred):
    return rpred[pred]

'''
Following two methods are for the readable target labels
'''
def get_rlabel():
    labels = {}
    for i, line in enumerate(open('labels.txt')):
        if i==0:
            continue
        elif i>1000:
            break
        words = line.split()
        cat = ' '.join(words[1:])
        labels[i] = cat
    return labels
rlabel = get_rlabel()

def readable_label(label):
    return rlabel[label]


# Now that we have an img, we need to preprocess it.
# We need to:
#       * resize the img, it is pretty big (~1200x1200px).
#       * normalize it, as noted in the PyTorch pretrained models doc,
#         with, mean = [0.485, 0.456, 0.406] and std = [0.229, 0.224, 0.225].
#       * convert it to a PyTorch Tensor.
#
# We can do all this preprocessing using a transform pipeline.
min_img_size = 224  # The min size, as noted in the PyTorch pretrained models doc, is 224 px.
transform_pipeline = transforms.Compose([transforms.Resize(min_img_size),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225])])
def preprocess(img):
    if img.mode=='L':
        img = img.convert("RGB")
    img = transform_pipeline(img)
    # PyTorch pretrained models expect the Tensor dims to be (num input imgs, num color channels, height, width).
    # Currently however, we have (num color channels, height, width); let's fix this by inserting a new axis.
    img = img.unsqueeze(0)  # Insert the new axis at index 0 i.e. in front of the other axes/dims. 
    # Now that we have preprocessed our img, we need to convert it into a 
    # Variable; PyTorch models expect inputs to be Variables. A PyTorch Variable is a  
    # wrapper around a PyTorch Tensor.
    img = Variable(img)
    return img

'''
get prediction given an image and the neural network
'''
def get_pred(nn, img):
    prediction = nn(img)  # Returns a Tensor of shape (batch, num class labels)
    prediction = prediction.data.numpy().argmax()  # Our prediction will be the index of the class label with the largest value.
    return prediction

'''
following two methods are to convert prediction label distribution to the target
label distribution
'''
def get_mapping():
    mapping = {}
    for pred_key, pred_val in rpred.items():
        for label_key, label_val in rlabel.items():
            if label_val==pred_val:
                mapping[pred_key] = label_key
                break
        if pred_key not in mapping:
            print(mapping)
            print(pred_val)
            assert pred_key in mapping
    return mapping
mapping = get_mapping()

def map_pred_to_label(pred):
    return mapping[pred]

'''
the method to get adversarial attack
'''
noise_scale=0.02
def get_adversarial_example(nn, img, output, target):
    criterion = torch.nn.CrossEntropyLoss()
    target = torch.LongTensor([target])
    loss = criterion(output, target)
    # retain_graph=True，calculate all grads of vars with requires_grad=True
    loss.backward(retain_graph=True)
    img_grad = torch.sign(img.grad.data)
    return img+noise_scale*img_grad

'''
The main loop to pred and attack
'''
def main_loop(nn):
    label_file = open('/content/ILSVRC2012_devkit_t12/data/ILSVRC2012_validation_ground_truth.txt')
    cnt, acc, fooled = 0, 0, 0
    size = len(os.listdir('imgs'))
    sample_gap = size//1000
    for label, img_file in zip(label_file, sorted(os.listdir('imgs'))):
        label = int(label)
        img = Image.open(f"imgs/{img_file}")
        # img = preprocess(img)
        if img.mode=='L':
            img = img.convert("RGB")
        img = transform_pipeline(img)
        img = img.unsqueeze(0)
        img = Variable(img, requires_grad=True)
        
        output = nn(img)
        pred = output.data.numpy().argmax()
        # pred_top2 = pred[1]
        # print(pred, type(pred), type(pred[0]))

        # label = label-1 #make it from 1~1000 to 0~999

        mapped_pred = map_pred_to_label(pred)
        cnt += 1
        if(map_pred_to_label(pred)==label):
            acc+=1
            attacked_img = get_adversarial_example(nn, img, output, pred)
            attacked_pred = get_pred(nn, attacked_img)
            if(map_pred_to_label(attacked_pred)!=label):
                fooled+=1
        if(cnt%sample_gap==0):
            print(f"{fooled}/{acc}/{cnt}/{size}, acc:{acc/cnt:.2f}, fooling rate:{0.0 if acc==0 else fooled/acc:.2f}")
            # print(pred, label)
            # print(img_file, pred, readable_pred(pred), label, readable_label(label))
    label_file.close()

# Let's get the cat img.
# response = requests.get(IMG_URL)
# img = Image.open(io.BytesIO(response.content))  # Read bytes and store as an img.
# print(img.size, "stop!")
# exit(0)
# Now let's load our model and get a prediciton!
vgg = models.vgg16(pretrained=True)  # This may take a few minutes.
main_loop(vgg)

FileNotFoundError: ignored

In [ ]:
!wget http://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar
!wget http://image-net.org/data/ILSVRC/2012/ILSVRC2012_devkit_t12.tar.gz

--2021-03-16 13:20:54--  http://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar
Resolving image-net.org (image-net.org)... 171.64.68.16
Connecting to image-net.org (image-net.org)|171.64.68.16|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6744924160 (6.3G) [application/x-tar]
Saving to: ‘ILSVRC2012_img_val.tar’

ILSVRC2012_img_val. 100%[===================>]   6.28G  4.13MB/s    in 31m 9s  

2021-03-16 13:52:03 (3.44 MB/s) - ‘ILSVRC2012_img_val.tar’ saved [6744924160/6744924160]

--2021-03-16 13:52:03--  http://image-net.org/data/ILSVRC/2012/ILSVRC2012_devkit_t12.tar.gz
Resolving image-net.org (image-net.org)... 171.64.68.16
Connecting to image-net.org (image-net.org)|171.64.68.16|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2568145 (2.4M) [application/x-gzip]
Saving to: ‘ILSVRC2012_devkit_t12.tar.gz’

ILSVRC2012_devkit_t 100%[===================>]   2.45M  2.43MB/s    in 1.0s    

2021-03-16 13:52:04 (2.43 MB/s) - ‘ILSVRC201

In [ ]:
from google.colab import drive
import shutil
drive.mount('/content/drive')
save_path = '/content/drive/My Drive/yuqing'
shutil.copy2('ILSVRC2012_img_val.tar', save_path)
shutil.copy2('ILSVRC2012_devkit_t12.tar.gz', save_path)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AY0e-g6QUgdcVft15lwT2uox36NFvsvOBOpYbnBbvbmCrTzj5IXZa-0thIM
Mounted at /content/drive


'/content/drive/My Drive/yuqing/ILSVRC2012_devkit_t12.tar.gz'

In [ ]:
!ls ILSVRC2012_devkit_t12.tar.gz/data

ls: cannot access 'ILSVRC2012_devkit_t12.tar.gz/data': Not a directory


In [ ]:
import torchvision.models as models
import torchvision
import json

with open('annotations/image_info_test2014.json') as f:
  data = json.load(f)
print(data.keys())
print(data['images'][0])
print(data['categories'][0])
# for image, c in zip(data['images'], data['categories']):
#     print(image['file_name'], c)
# exit(0)
# vgg = models.vgg16(pretrained=True)
# imagenet_data = torchvision.datasets.ImageNet('path/to/imagenet_root/')
# data_loader = torch.utils.data.DataLoader(imagenet_data,
#                     batch_size=4,
#                     shuffle=True,
#                     num_workers=args.nThreads)


dict_keys(['info', 'images', 'licenses', 'categories'])
{'license': 2, 'file_name': 'COCO_test2014_000000523573.jpg', 'coco_url': 'http://images.cocodataset.org/test2014/COCO_test2014_000000523573.jpg', 'height': 500, 'width': 423, 'date_captured': '2013-11-14 12:21:59', 'id': 523573}
{'supercategory': 'person', 'id': 1, 'name': 'person'}
